<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Key2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/MarkSecada/key2vec.git

Cloning into 'key2vec'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (75/75), done.


In [ ]:

!git clone https://github.com/uclnlp/inferbeddings/blob/master/data/glove/glove.6B.50d.txt.gz

Cloning into 'glove.6B.50d.txt.gz'...
fatal: repository 'https://github.com/uclnlp/inferbeddings/blob/master/data/glove/glove.6B.50d.txt.gz/' not found


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:


!python /content/key2vec/test.py